In [4]:
#Running all console commands for install
%pip install dash
%pip install plotly
%pip install wget

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
#importing all the packages needed
import dash
from dash import dcc, html, callback, Output, Input
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import requests
import io

In [3]:
#import the data
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv"
response = requests.get(URL)
data = pd.read_csv(URL)

In [6]:
app = dash.Dash(__name__)

year_list = [i for i in range(1980, 2024)]

#making the layout for the dashboard
app.layout = html.Div([
    html.H1(
        children='Automobile Sales Statistics Dashboard',
        style={
            'textAlign': 'center',
            'color': '#503D36',
            'fontSize': '24px'
        }
    ),
    
    
    #making the dropdown box for report type
    dcc.Dropdown(
        id='dropdown-statistics',
        options=[
            {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
            {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
        ],
        placeholder='Select a report type',
        value='Select Statistics',
        style={
            'width': '80%',
            'padding': '3px',
            'fontSize': '20px',
            'textAlignLast': 'center'
        }
    ),
    
    
    #making dropdown to select year
    dcc.Dropdown(
        id='select-year',
        options=[{'label': i, 'value': i} for i in year_list],
        placeholder='Select year',
        value='Select-year',
        style={
            'width': '80%',
            'padding': '3px',
            'fontSize': '20px',
            'textAlignLast': 'center'
        }
    ),
    
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'})
    ])
])

#functions returned to create the plots
def create_yearly_plot1(yearly_data):
    
    #creates line graph for sales over time
    fig = px.line(yearly_data,
                  x='Month',
                  y='Automobile_Sales',
                  title=f'Automobile Sales in {yearly_data["Year"].iloc[0]}')
    return fig


def create_yearly_plot2(yearly_data):
    
    #creates bar chart for average sales by vehicle type
    avg_sales = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
    fig = px.bar(avg_sales,
                 x='Vehicle_Type',
                 y='Automobile_Sales',
                 title=f'Average Automobile Sales by Vehicle Type in {yearly_data["Year"].iloc[0]}')
    return fig


def create_yearly_plot3(yearly_data):
    
    #pie chart for total expenses by vehicle type
    total_sales = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].sum().reset_index()
    fig = px.pie(total_sales,
                 values='Automobile_Sales',
                 names='Vehicle_Type',
                 title=f'Total Sales Share by Vehicle Type in {yearly_data["Year"].iloc[0]}')
    return fig


def create_yearly_plot4(yearly_data):
    
    #unemployement rate compared to sales by vehicle type
    ad_exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
    fig = px.bar(ad_exp_data,
                 x='Vehicle_Type',
                 y='Advertising_Expenditure',
                 title=f'Advertising Expenditure by Vehicle Type in {yearly_data["Year"].iloc[0]}')
    return fig

#callback component
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='select-year', component_property='value')]
)

#this is the output container that is showing the plots 
def update_output_container(selected_statistic, selected_year):
    if selected_statistic == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]

        #first plot: line graph
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec,
                           x='Year',
                           y='Automobile_Sales',
                           title="Automobile Sales Fluctuations Over Recession Period")
        )

        #second plot: bar graph
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        R_chart2 = dcc.Graph(
            figure=px.bar(average_sales,
                          x='Vehicle_Type',
                          y='Automobile_Sales',
                          title="Average Number of Vehicles Sold by Vehicle Type")
        )

        #third plot: pie chart
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec,
                          values='Advertising_Expenditure',
                          names='Vehicle_Type',
                          title="Total Expenditure Share by Vehicle Type During Recessions")
        )

        #fourth plot: line graph
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure=px.bar(unemp_data,
                          x='unemployment_rate',
                          y='Automobile_Sales',
                          color='Vehicle_Type',
                          labels={'unemployment_rate': 'Unemployment Rate', 'Automobile_Sales': 'Average Automobile Sales'},
                          title='Effect of Unemployment Rate on Vehicle Type and Sales')
        )

        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1), html.Div(children=R_chart2)], style={'display': 'flex'}),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3), html.Div(children=R_chart4)], style={'display': 'flex'})
        ]

    elif selected_statistic == 'Yearly Statistics':
        if selected_year and selected_year != 'Select-year':
            yearly_data = data[data['Year'] == selected_year]

            plots = [
                dcc.Graph(figure=create_yearly_plot1(yearly_data)),
                dcc.Graph(figure=create_yearly_plot2(yearly_data)),
                dcc.Graph(figure=create_yearly_plot3(yearly_data)),
                dcc.Graph(figure=create_yearly_plot4(yearly_data))
            ]

            return html.Div(children=plots, style={'display': 'grid', 'grid-template-columns': '1fr 1fr', 'gap': '20px'})

        return html.Div([
            html.H3('Please select a valid year to view yearly statistics.', style={'textAlign': 'center'})
        ])

    return html.Div([
        html.H3('Please select a valid report type to view the statistics.', style={'textAlign': 'center'})
    ])

if __name__ == '__main__':
    app.run_server(port=8090, debug=True)
